In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='2f756608-0d9d-4f27-ac44-84d3e0bf814e', project_access_token='p-c7ddbe2c2977a3c8fb8b908eb21edd0c9048f1fb')
pc = project.project_context


# MARATONA BEHIND THE CODE 2020

## DESAFIO 6 - ANAHUAC

### Introducción

En este desafio, usted usará herramientas de IBM como Watson Studio (o Cloud Pak for Data) para construir un modelo baseado en Machine Learning capaz de preveer si un estudante irá continuar o abandonará su curso.

<hr>

## Installing Libs

In [2]:
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.23.2)


In [3]:
!pip install xgboost --upgrade

Requirement already up-to-date: xgboost in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.2.0)


<hr>

## Loading the .csv dataset from GitHub

In [4]:
import pandas as pd

In [5]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/ForTraining.csv
df_base_for_training = pd.read_csv(r'ForTraining.csv')
df_base_for_training.head(10)

--2020-09-04 08:13:13--  https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/ForTraining.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92376 (90K) [text/plain]
Saving to: ‘ForTraining.csv.2’

100%[======================================>] 92,376      --.-K/s   in 0.006s  

2020-09-04 08:13:13 (15.4 MB/s) - ‘ForTraining.csv.2’ saved [92376/92376]



,studentId,reducido,ciclo,Graduado
0,88588.0,Interv. Edu.,2019 - 08,No
1,81135.0,Educacion,2019 - 05,Si
2,80926.0,Direc. Estrat. Neg,2019 - 05,Si
3,82415.0,B. I.,2019 - 06,Si
4,90082.0,Est. Temprana,2019 - 08,No
5,51963.0,Mercadotecnia,2017 - 06,No
6,79559.0,E. Lectoescritura,2019 - 05,No
7,76347.0,Des. Neg. y Empren.,2019 - 03,Si
8,71147.0,Neuromarketing,2019 - 01,No
9,79108.0,Programacion de Apps,2019 - 05,No


Descripción: La primera tabla mostrada arriba tiene 4 columnas, 3 son features and el target: `Graduado` that has a binary values={Si, No}.

Usted puede, y debe, usar mas data que esta disponible para construir su modelo. Los siguientes archivos .csv presentados:

In [6]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/OrdenMaterias.csv
df_orden_materias = pd.read_csv(r'OrdenMaterias.csv')
df_orden_materias.head()

--2020-09-04 08:13:14--  https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/OrdenMaterias.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63920 (62K) [text/plain]
Saving to: ‘OrdenMaterias.csv.2’

100%[======================================>] 63,920      --.-K/s   in 0.003s  

2020-09-04 08:13:14 (19.5 MB/s) - ‘OrdenMaterias.csv.2’ saved [63920/63920]



,reducido,2017 - 03,2017 - 04,2017 - 05,2017 - 06,2017 - 07,2017 - 08,2018 - 01,2018 - 02,2018 - 03,...,2019 - 05,2019 - 06,2019 - 07,2019 - 08,2020 - 01,2020 - 02,2020 - 03,2020 - 04,2020 - 05,2020 - 06
0,Act. Derecho Contratual,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Sin clase,Sin clase,Sin clase,Sin clase,Consentimiento y objeto digital contractual,Métodos alternos de resolución de controversias,Nueva clasificación contractual,Nuevo derecho probatorio documental,Contratos inteligentes,Consentimiento y objeto digital contractual
1,Adm. Emp.,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Mercadotecnia Estratégica,Contabilidad Financiera y Presupuestos,Administración de costos,Impuestos,Desafíos actuales en la Dirección del Capital ...,Mercadotecnia Estratégica,Contabilidad Financiera y Presupuestos,Administración de costos,Impuestos,Desafíos actuales en la Dirección del Capital ...
2,Adm. Riesgos,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Riesgos empresariales,Contabilidad Financiera y Presupuestos,Administración de riesgos,Evaluación de Riesgos,Toma de Decisiones,Riesgos empresariales,Contabilidad Financiera y Presupuestos,Administración de riesgos,Evaluación de Riesgos,Toma de Decisiones
3,Adm. Salud,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Calidad en salud,Contabilidad Financiera y Presupuestos,Administración de costos,Salud pública,Planeación estratégica en salud,Calidad en salud,Contabilidad Financiera y Presupuestos,Planeación estratégica en salud,Salud pública,Planeación estratégica en salud
4,Admon Fin.,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Decisión y Análisis de Inversión,Decisión de Financiamiento y Pago de Dividendos,Análisis de Estados Financieros,Estrategias de negocios,Planeación Financiera y Administración del Cap...,Decisión y Análisis de Inversión,Decisión de Financiamiento y Pago de Dividendos,Análisis de Estados Financieros,Estrategias de negocios,Planeación Financiera y Administración del Cap...


In [7]:
df_orden_materias.tail()

,reducido,2017 - 03,2017 - 04,2017 - 05,2017 - 06,2017 - 07,2017 - 08,2018 - 01,2018 - 02,2018 - 03,...,2019 - 05,2019 - 06,2019 - 07,2019 - 08,2020 - 01,2020 - 02,2020 - 03,2020 - 04,2020 - 05,2020 - 06
110,Transf. Globales,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Sin clase,Sin clase,Tendencia de las organizaciones,Sin clase,Sociedad de la Información,Mercados globales y consecuencias,Mercados globales y consecuencias,Paz y Seguridad Internacionales,Tendencia de las organizaciones,Sociedad de la Información
111,Transparencia,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Sin clase,Sin clase,Sin clase,Sin clase,Generalidades y clasificación de la corrupcción,Generalidades y clasificación de la corrupcción,La actuación ética y responsabilidad administr...,Conductas delictivas asociadas a la corrupción,Corrupción en las empresas,El Sistema Nacional y Estatal Anticorrupción
112,Trastornos Alimen.,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Sin clase,Epidemiología de los TCA y la obesidad. Cultur...,Factores de riesgo / Conductas de riesgo. Inte...,Diagnóstico de los TCA y la obesidad,Tratamiento de los TCA. Manejo integral y mult...,Prevención,Epidemiología de los TCA y la obesidad. Cultur...,Factores de riesgo / Conductas de riesgo. Inte...,Diagnóstico de los TCA y la obesidad,Tratamiento de los TCA. Manejo integral y mult...
113,Ventas Estrat.,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Mercadotecnia Estratégica,Contabilidad Financiera y Presupuestos,Liderazgo y Motivación,Visión Comercial,Excelencia en el Servicio al Cliente,Mercadotecnia Estratégica,Contabilidad Financiera y Presupuestos,Liderazgo y Motivación,Visión Comercial,Excelencia en el Servicio al Cliente
114,Vida Indep.,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,Sin clase,...,Sin clase,Sin clase,Sin clase,Sin clase,Autodeterminacion/autogestor,Familia,Inclusión,Marco Normativo,Discapacidad,Autodeterminacion/autogestor


In [8]:
df_orden_materias[df_orden_materias['reducido']=='Educacion']

,reducido,2017 - 03,2017 - 04,2017 - 05,2017 - 06,2017 - 07,2017 - 08,2018 - 01,2018 - 02,2018 - 03,...,2019 - 05,2019 - 06,2019 - 07,2019 - 08,2020 - 01,2020 - 02,2020 - 03,2020 - 04,2020 - 05,2020 - 06
47,Educacion,La Tecnología en la Educación,Crecimiento y Desarrollo Humano,Formación Integral,Estrategias de Enseñanza,Evaluación del Aprendizaje,La Tecnología en la Educación,Crecimiento y Desarrollo Humano,Formación Integral,Estrategias de Enseñanza,...,Estrategias de Enseñanza,Evaluación del Aprendizaje,La Tecnología en la Educación,Crecimiento y Desarrollo Humano,Formación Integral,Estrategias de Enseñanza,Evaluación del Aprendizaje,La Tecnología en la Educación,Crecimiento y Desarrollo Humano,Formación Integral


In [9]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/TablaConexiones.csv
df_tabla_conexiones = pd.read_csv(r'TablaConexiones.csv')
df_tabla_conexiones.head()

--2020-09-04 08:13:15--  https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/TablaConexiones.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243492 (238K) [text/plain]
Saving to: ‘TablaConexiones.csv.2’

100%[======================================>] 243,492     --.-K/s   in 0.01s   

2020-09-04 08:13:15 (23.1 MB/s) - ‘TablaConexiones.csv.2’ saved [243492/243492]



,studentId,ciclo,Dias_Conectado,Minutos_Promedio,Minutos_Total
0,45870,2017 - 03,10.0,76.957000,769.57
1,45898,2017 - 03,32.0,370.231250,11847.40
2,45905,2017 - 03,50.0,198.717400,9935.87
3,45931,2017 - 03,22.0,294.535909,6479.79
4,45933,2017 - 03,18.0,177.991667,3203.85


In [10]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/TablaTareas.csv
df_tabla_tareas = pd.read_csv(r'TablaTareas.csv')
df_tabla_tareas.head()

--2020-09-04 08:13:16--  https://raw.githubusercontent.com/vanderlei-test/dataset2/master/datasets/TablaTareas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310923 (304K) [text/plain]
Saving to: ‘TablaTareas.csv.2’

100%[======================================>] 310,923     --.-K/s   in 0.01s   

2020-09-04 08:13:16 (28.1 MB/s) - ‘TablaTareas.csv.2’ saved [310923/310923]



,studentId,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas
0,81328.0,2019 - 05,99.000000,10.0,0.0,0.0,10.0
1,99206.0,2020 - 02,80.777500,10.0,2.0,0.0,12.0
2,94582.0,2020 - 01,79.666000,10.0,0.0,0.0,10.0
3,87564.0,2019 - 07,0.000000,0.0,15.0,0.0,15.0
4,87971.0,2019 - 08,36.363636,5.0,6.0,0.0,11.0


In [11]:
df_si = df_base_for_training[df_base_for_training['Graduado']=='Si']

In [12]:
df_no =df_base_for_training[df_base_for_training['Graduado']=='No']

In [13]:
df_si.describe()

,studentId
count,1280.000000
mean,76192.678906
std,10627.031391
min,45898.000000
25%,70708.250000
50%,78619.500000
75%,83984.000000
max,91335.000000


In [14]:
df_no.describe()

,studentId
count,1220.000000
mean,77491.125410
std,10393.302772
min,45905.000000
25%,73052.250000
50%,79595.000000
75%,85244.750000
max,91346.000000


In [15]:
df_tabla_tareas[df_tabla_tareas['studentId']==81135.0]

,studentId,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas
1935,81135.0,2019 - 05,90.0,10.0,0.0,0.0,10.0


In [16]:
df_tabla_conexiones[df_tabla_conexiones['studentId']==81135.0]

,studentId,ciclo,Dias_Conectado,Minutos_Promedio,Minutos_Total
2644,81135,2019 - 05,182.0,328.250934,59741.67


Overview del Dataset:

    Disponibles para el participante, ecisten 4 tables cargas en DataFrames anteriormente:
    
    **df_base_for_training**
        - ``studentId``
        ``reducido``
        ``ciclo``
        ``Graduado`` --> ¡LA VARIABLE OBJETIVO PARA CLASIFICACIÓN BINARIA!
        
    **df_orden_materias**
        ``reducido``
        ``2017 - 03``
        ``2017 - 04``
        ``2017 - 05``
        ``2017 - 06``
        ``2017 - 07``
        ``2017 - 08``
        ``2018 - 01``
        ``2018 - 02``
        ``2018 - 03``
        ``2018 - 04``
        ``2018 - 05``
        ``2018 - 06``
        ``2018 - 07``
        ``2018 - 08``
        ``2019 - 01``
        ``2019 - 02``
        ``2019 - 03``
        ``2019 - 04``
        ``2019 - 05``
        ``2019 - 06``
        ``2019 - 07``
        ``2019 - 08``
        ``2020 - 01``
        ``2020 - 02``
        ``2020 - 03``
        ``2020 - 04``
        ``2020 - 05``
        ``2020 - 06``
        
    **df_tabla_conexiones**
        - ``studentId``
        ``ciclo``
        ``Dias_Conectado``
        ``Minutos_Promedio``
        ``Minutos_Total``
        
    **df_tabla_tareas**
        - ``studentId``
        ``ciclo``
        ``Calificacion_Promedio``
        ``Tareas_Puntuales``
        ``Tareas_No_Entregadas``
        ``Tareas_Retrasadas``
        ``Total_Tareas``
        
Observe que la variable ``studentId`` aparece en varias tablas.

Usted puede combinar/merge estos datasets como usted desee.

In [17]:
print("Columnas en *df_base_for_training*:")
print(df_base_for_training.columns)

print("\Columnas en *df_orden_materias*:")
print(df_orden_materias.columns)

print("\Columnas en *df_tabla_conexiones*:")
print(df_tabla_conexiones.columns)

print("\Columnas en *df_tabla_tareas*:")
print(df_tabla_tareas.columns)

Columnas en *df_base_for_training*:
Index(['studentId', 'reducido', 'ciclo', 'Graduado'], dtype='object')
\Columnas en *df_orden_materias*:
Index(['reducido', '2017 - 03', '2017 - 04', '2017 - 05', '2017 - 06',
       '2017 - 07', '2017 - 08', '2018 - 01', '2018 - 02', '2018 - 03',
       '2018 - 04', '2018 - 05', '2018 - 06', '2018 - 07', '2018 - 08',
       '2019 - 01', '2019 - 02', '2019 - 03', '2019 - 04', '2019 - 05',
       '2019 - 06', '2019 - 07', '2019 - 08', '2020 - 01', '2020 - 02',
       '2020 - 03', '2020 - 04', '2020 - 05', '2020 - 06'],
      dtype='object')
\Columnas en *df_tabla_conexiones*:
Index(['studentId', 'ciclo', 'Dias_Conectado', 'Minutos_Promedio',
       'Minutos_Total'],
      dtype='object')
\Columnas en *df_tabla_tareas*:
Index(['studentId', 'ciclo', 'Calificacion_Promedio', 'Tareas_Puntuales',
       'Tareas_No_Entregadas', 'Tareas_Retrasadas', 'Total_Tareas'],
      dtype='object')


In [18]:
print("Información de *df_base_for_training*:")
print(df_base_for_training.info())

Información de *df_base_for_training*:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 4 columns):
studentId    2500 non-null float64
reducido     2500 non-null object
ciclo        2500 non-null object
Graduado     2500 non-null object
dtypes: float64(1), object(3)
memory usage: 78.2+ KB
None


In [19]:
print("\Información de *df_orden_materias*:")
print(df_orden_materias.info())


\Información de *df_orden_materias*:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 29 columns):
reducido     115 non-null object
2017 - 03    115 non-null object
2017 - 04    115 non-null object
2017 - 05    115 non-null object
2017 - 06    115 non-null object
2017 - 07    115 non-null object
2017 - 08    115 non-null object
2018 - 01    115 non-null object
2018 - 02    115 non-null object
2018 - 03    115 non-null object
2018 - 04    115 non-null object
2018 - 05    115 non-null object
2018 - 06    115 non-null object
2018 - 07    115 non-null object
2018 - 08    115 non-null object
2019 - 01    115 non-null object
2019 - 02    115 non-null object
2019 - 03    115 non-null object
2019 - 04    115 non-null object
2019 - 05    115 non-null object
2019 - 06    115 non-null object
2019 - 07    115 non-null object
2019 - 08    115 non-null object
2020 - 01    115 non-null object
2020 - 02    115 non-null object
2020 - 03    115 non-null object


In [20]:

print("\Información de *df_tabla_conexiones*:")
print(df_tabla_conexiones.info())



\Información de *df_tabla_conexiones*:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4496 entries, 0 to 4495
Data columns (total 5 columns):
studentId           4496 non-null int64
ciclo               4496 non-null object
Dias_Conectado      4496 non-null float64
Minutos_Promedio    4496 non-null float64
Minutos_Total       4496 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 175.7+ KB
None


In [21]:
print("\Información de *df_tabla_tareas*:")
print(df_tabla_tareas.info())

\Información de *df_tabla_tareas*:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6298 entries, 0 to 6297
Data columns (total 7 columns):
studentId                6298 non-null float64
ciclo                    6298 non-null object
Calificacion_Promedio    6298 non-null float64
Tareas_Puntuales         6298 non-null float64
Tareas_No_Entregadas     6298 non-null float64
Tareas_Retrasadas        6298 non-null float64
Total_Tareas             6298 non-null float64
dtypes: float64(6), object(1)
memory usage: 344.5+ KB
None


In [22]:
print("Descripción de *df_base_for_training*:")
df_base_for_training.describe().transpose()


Descripción de *df_base_for_training*:


,count,mean,std,min,25%,50%,75%,max
studentId,2500.0,76826.3208,10531.545832,45898.0,71813.25,79062.5,84517.25,91346.0


In [23]:
print("\Descripción de *df_orden_materias*:")
df_orden_materias.describe().transpose()


\Descripción de *df_orden_materias*:


,count,unique,top,freq
reducido,115,115,Analitica Web,1
2017 - 03,115,4,Sin clase,107
2017 - 04,115,8,Sin clase,105
2017 - 05,115,8,Sin clase,103
2017 - 06,115,9,Sin clase,103
2017 - 07,115,9,Sin clase,103
2017 - 08,115,4,Sin clase,103
2018 - 01,115,8,Sin clase,103
2018 - 02,115,8,Sin clase,102
2018 - 03,115,9,Sin clase,102


In [24]:
print("\Descripción de *df_tabla_conexiones*:")
df_tabla_conexiones.describe().transpose()

\Descripción de *df_tabla_conexiones*:


,count,mean,std,min,25%,50%,75%,max
studentId,4496.0,76837.757340,10483.487462,45870.0,71747.500000,79145.000000,84520.750000,91437.00
Dias_Conectado,4496.0,49.265347,33.047478,1.0,24.000000,43.000000,71.000000,199.00
Minutos_Promedio,4496.0,256.258709,147.012273,0.0,155.581844,228.662358,327.788481,1215.70
Minutos_Total,4496.0,12100.932931,9877.737441,0.0,5080.940000,9889.530000,16493.202500,116177.03


In [25]:
print("\Descripción de *df_tabla_tareas*:")
df_tabla_tareas.describe().transpose()

\Descripción de *df_tabla_tareas*:


,count,mean,std,min,25%,50%,75%,max
studentId,6298.0,82465.858844,12750.589042,45870.0,75283.250000,83396.000000,92447.750,102590.0
Calificacion_Promedio,6298.0,70.491830,28.568343,0.0,63.815357,81.538077,90.222,100.0
Tareas_Puntuales,6298.0,8.909495,4.113840,0.0,7.000000,10.000000,11.000,29.0
Tareas_No_Entregadas,6298.0,2.325500,3.878632,0.0,0.000000,0.000000,3.000,34.0
Tareas_Retrasadas,6298.0,0.185297,0.642416,0.0,0.000000,0.000000,0.000,11.0
Total_Tareas,6298.0,11.420292,3.158823,5.0,10.000000,11.000000,14.000,34.0


#### ¡ATENCIÓN! La columna **target** es  ``Graduado``, presente en el DataFrame "df_base_for_training".

<hr>

In [26]:
print("Valores únicos de *df_base_for_training*:")
df_base_for_training.nunique(axis = 0)

Valores únicos de *df_base_for_training*:


studentId    2500
reducido       88
ciclo          22
Graduado        2
dtype: int64

In [27]:
print("Valores únicos de *df_orden_materias*:")
df_orden_materias.nunique(axis = 0)

Valores únicos de *df_orden_materias*:


reducido     115
2017 - 03      4
2017 - 04      8
2017 - 05      8
2017 - 06      9
2017 - 07      9
2017 - 08      4
2018 - 01      8
2018 - 02      8
2018 - 03      9
2018 - 04      9
2018 - 05      4
2018 - 06      8
2018 - 07     10
2018 - 08     14
2019 - 01     14
2019 - 02     17
2019 - 03     24
2019 - 04     32
2019 - 05     34
2019 - 06     37
2019 - 07     58
2019 - 08     58
2020 - 01     72
2020 - 02     69
2020 - 03     64
2020 - 04     75
2020 - 05     83
2020 - 06     85
dtype: int64

In [28]:
print("Valores únicos de *df_tabla_conexiones*:")
df_tabla_conexiones.nunique(axis = 0)

Valores únicos de *df_tabla_conexiones*:


studentId           4496
ciclo                 22
Dias_Conectado       167
Minutos_Promedio    4484
Minutos_Total       4479
dtype: int64

In [29]:
print("Valores únicos de *df_tabla_tareas*:")
df_tabla_tareas.nunique(axis = 0)

Valores únicos de *df_tabla_tareas*:


studentId                6298
ciclo                      24
Calificacion_Promedio    3277
Tareas_Puntuales           25
Tareas_No_Entregadas       30
Tareas_Retrasadas          11
Total_Tareas               22
dtype: int64

In [30]:
print("Valores distintos de *df_orden_materias*:")
df_orden_materias["2017 - 03"].value_counts()

Valores distintos de *df_orden_materias*:


Sin clase                        107
Liderazgo y Motivación             5
La Tecnología en la Educación      2
Finanzas para no Financieros       1
Name: 2017 - 03, dtype: int64

In [31]:
print("Valores distintos de *df_orden_materias*:")
df_orden_materias["2020 - 06"].value_counts()

Valores distintos de *df_orden_materias*:


Toma de Decisiones                                                                       5
Nuevos modelos de negocios                                                               4
Metodologías activas, aprendizaje situado y didáctica diferencial                        4
Trabajo en Equipo                                                                        4
Data Driven Decisions                                                                    3
Inteligencia Emocional                                                                   3
Sin clase                                                                                3
Componentes básicos en Inteligencia Artificial                                           2
Programación en Python                                                                   2
Evaluación del Aprendizaje                                                               2
Comunicación corporativa                                                                 2

In [32]:
print("Valores más frecuentes de *df_orden_materias*:")
df_orden_materias["2020 - 06"].mode()

Valores más frecuentes de *df_orden_materias*:


0    Toma de Decisiones
dtype: object

In [33]:
## Graduados vs no graduados por ciclo

df_base_for_training.groupby(['Graduado']).size().reset_index(name='counts')

,Graduado,counts
0,No,1220
1,Si,1280


In [34]:
## Graduados vs no graduados por ciclo

df_base_for_training.groupby(['ciclo','Graduado']).size().reset_index(name='counts')

,ciclo,Graduado,counts
0,2017 - 03,No,7
1,2017 - 03,Si,8
2,2017 - 04,No,9
3,2017 - 04,Si,13
4,2017 - 05,No,15
5,2017 - 05,Si,19
6,2017 - 06,No,11
7,2017 - 06,Si,11
8,2017 - 07,No,11
9,2017 - 07,Si,24


#### Transformaciones

##### df_orden_materias

In [35]:
#Añadir esta columna -- Num ciclos del programa (reducido)
print("Valores distintos de cada programa *df_orden_materias*:")
df_orden_materias.nunique(axis=1)-1

Valores distintos de cada programa *df_orden_materias*:


0      6
1      6
2      6
3      6
4      6
5      6
6      6
7      4
8      6
9      6
10     6
11     6
12     6
13     6
14     6
15     6
16     7
17     6
18     6
19     6
20     6
21     6
22     4
23     3
24     5
25     6
26     6
27     6
28     6
29     5
      ..
85     7
86     6
87     6
88     6
89     6
90     6
91     7
92     6
93     6
94     6
95     5
96     6
97     6
98     6
99     6
100    6
101    6
102    6
103    7
104    6
105    5
106    2
107    6
108    6
109    6
110    5
111    6
112    6
113    6
114    6
Length: 115, dtype: int64

In [36]:
df_orden_materias["num_modulos"] = df_orden_materias.nunique(axis=1)-2

In [37]:
df_modulos = df_orden_materias[["reducido","num_modulos"]]

In [38]:
df_modulos

,reducido,num_modulos
0,Act. Derecho Contratual,5
1,Adm. Emp.,5
2,Adm. Riesgos,5
3,Adm. Salud,5
4,Admon Fin.,5
5,Alim. Y Nut. Salud,5
6,Analitica Web,5
7,Autolesion,3
8,B. I.,5
9,Bien. Salud y Calidad,5


##### df_tabla_tareas

In [39]:
df_tabla_tareas.head()

,studentId,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas
0,81328.0,2019 - 05,99.000000,10.0,0.0,0.0,10.0
1,99206.0,2020 - 02,80.777500,10.0,2.0,0.0,12.0
2,94582.0,2020 - 01,79.666000,10.0,0.0,0.0,10.0
3,87564.0,2019 - 07,0.000000,0.0,15.0,0.0,15.0
4,87971.0,2019 - 08,36.363636,5.0,6.0,0.0,11.0


In [40]:
df_tabla_tareas["prop_tareas_puntuales"]= df_tabla_tareas["Tareas_Puntuales"]/df_tabla_tareas["Total_Tareas"]
df_tabla_tareas["prop_tareas_retrasadas"]= df_tabla_tareas["Tareas_Retrasadas"]/df_tabla_tareas["Total_Tareas"]
df_tabla_tareas["prop_tareas_no_entregadas"]= df_tabla_tareas["Tareas_No_Entregadas"]/df_tabla_tareas["Total_Tareas"]

In [41]:
df_tabla_tareas.head()

,studentId,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas,prop_tareas_puntuales,prop_tareas_retrasadas,prop_tareas_no_entregadas
0,81328.0,2019 - 05,99.000000,10.0,0.0,0.0,10.0,1.000000,0.0,0.000000
1,99206.0,2020 - 02,80.777500,10.0,2.0,0.0,12.0,0.833333,0.0,0.166667
2,94582.0,2020 - 01,79.666000,10.0,0.0,0.0,10.0,1.000000,0.0,0.000000
3,87564.0,2019 - 07,0.000000,0.0,15.0,0.0,15.0,0.000000,0.0,1.000000
4,87971.0,2019 - 08,36.363636,5.0,6.0,0.0,11.0,0.454545,0.0,0.545455


In [42]:
## Pruebas con más columnas
##df_prop_tareas = df_tabla_tareas.drop(["Tareas_Puntuales","Tareas_No_Entregadas","Tareas_Retrasadas","Total_Tareas"],axis=1)
df_prop_tareas = df_tabla_tareas

In [43]:
df_prop_tareas.tail()

,studentId,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas,prop_tareas_puntuales,prop_tareas_retrasadas,prop_tareas_no_entregadas
6293,80467.0,2019 - 05,20.000000,1.0,4.0,0.0,5.0,0.200000,0.0,0.800000
6294,80249.0,2019 - 05,50.600000,3.0,2.0,0.0,5.0,0.600000,0.0,0.400000
6295,85857.0,2019 - 07,81.428571,13.0,1.0,0.0,14.0,0.928571,0.0,0.071429
6296,98880.0,2020 - 02,91.212727,11.0,0.0,0.0,11.0,1.000000,0.0,0.000000
6297,71859.0,2019 - 02,87.333000,10.0,0.0,0.0,10.0,1.000000,0.0,0.000000


## Uniendo DataFrames en Pandas

Documentación oficial para Pandas 1.1.0: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

Como un **ejemplo** de como usar Pandas, camos a unir/merge la información de las tablas "df_base_for_training" y "df_tabla_tareas" a traves de la llave ``studentId``.

Usted puee editar el dataframes manualmente si lo prefiere, usando Microsoft Excel u otros lenguajes. Recuerde insertar la data procesada en IBM Cloud Pak for Data para que pueda entrenar su modelo.

In [44]:
df2 = pd.merge(df_base_for_training,df_tabla_conexiones,how='inner')
df2.tail()

,studentId,reducido,ciclo,Graduado,Dias_Conectado,Minutos_Promedio,Minutos_Total
2495,89685.0,Estrat. Comercial,2019 - 08,No,56.0,162.077500,9076.34
2496,77590.0,Crianza Hijos,2019 - 06,Si,36.0,446.748333,16082.94
2497,57623.0,Mercadotecnia,2018 - 02,Si,71.0,131.249155,9318.69
2498,50018.0,Lider. Inst. Edu.,2017 - 05,No,7.0,140.868571,986.08
2499,88250.0,Prof. Deportiva.,2019 - 08,No,34.0,651.966765,22166.87


In [45]:
df3 = df2.merge(df_modulos,how = 'left')
df3.tail()

,studentId,reducido,ciclo,Graduado,Dias_Conectado,Minutos_Promedio,Minutos_Total,num_modulos
2495,89685.0,Estrat. Comercial,2019 - 08,No,56.0,162.077500,9076.34,6
2496,77590.0,Crianza Hijos,2019 - 06,Si,36.0,446.748333,16082.94,5
2497,57623.0,Mercadotecnia,2018 - 02,Si,71.0,131.249155,9318.69,4
2498,50018.0,Lider. Inst. Edu.,2017 - 05,No,7.0,140.868571,986.08,5
2499,88250.0,Prof. Deportiva.,2019 - 08,No,34.0,651.966765,22166.87,5


In [46]:
df4 = df3.loc[:, ["studentId","reducido","Graduado","Dias_Conectado",	"Minutos_Promedio",	"Minutos_Total","num_modulos"]]
df4.tail(15)

,studentId,reducido,Graduado,Dias_Conectado,Minutos_Promedio,Minutos_Total,num_modulos
2485,77676.0,Fin. Corp. Avan.,Si,65.0,210.208154,13663.53,5
2486,80378.0,Direc. Emp. Fam.,No,4.0,543.842500,2175.37,5
2487,75290.0,Alim. Y Nut. Salud,Si,94.0,218.178191,20508.75,5
2488,82936.0,Coaching de Vida,Si,72.0,287.149722,20674.78,5
2489,73904.0,Big Data,Si,66.0,275.751515,18199.60,5
2490,77671.0,Alim. Y Nut. Salud,Si,34.0,451.847059,15362.80,5
2491,84572.0,Com. Corp.,Si,75.0,132.267467,9920.06,5
2492,76266.0,Direc. Proy.,Si,38.0,175.453947,6667.25,4
2493,82090.0,Adm. Salud,Si,51.0,507.780000,25896.78,5
2494,68358.0,Big Data,Si,134.0,385.488507,51655.46,5


In [47]:
df5 = pd.merge(df4,df_prop_tareas,how = 'left')
df5.tail(15)

,studentId,reducido,Graduado,Dias_Conectado,Minutos_Promedio,Minutos_Total,num_modulos,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas,prop_tareas_puntuales,prop_tareas_retrasadas,prop_tareas_no_entregadas
2485,77676.0,Fin. Corp. Avan.,Si,65.0,210.208154,13663.53,5,2019 - 04,94.242000,5.0,0.0,0.0,5.0,1.0,0.0,0.0
2486,80378.0,Direc. Emp. Fam.,No,4.0,543.842500,2175.37,5,2019 - 05,8.000000,1.0,9.0,0.0,10.0,0.1,0.0,0.9
2487,75290.0,Alim. Y Nut. Salud,Si,94.0,218.178191,20508.75,5,2019 - 03,89.594667,15.0,0.0,0.0,15.0,1.0,0.0,0.0
2488,82936.0,Coaching de Vida,Si,72.0,287.149722,20674.78,5,2019 - 06,92.857143,14.0,0.0,0.0,14.0,1.0,0.0,0.0
2489,73904.0,Big Data,Si,66.0,275.751515,18199.60,5,2019 - 02,99.000000,10.0,0.0,0.0,10.0,1.0,0.0,0.0
2490,77671.0,Alim. Y Nut. Salud,Si,34.0,451.847059,15362.80,5,2019 - 04,88.666667,15.0,0.0,0.0,15.0,1.0,0.0,0.0
2491,84572.0,Com. Corp.,Si,75.0,132.267467,9920.06,5,2019 - 07,92.334000,10.0,0.0,0.0,10.0,1.0,0.0,0.0
2492,76266.0,Direc. Proy.,Si,38.0,175.453947,6667.25,4,2019 - 03,93.703333,9.0,0.0,0.0,9.0,1.0,0.0,0.0
2493,82090.0,Adm. Salud,Si,51.0,507.780000,25896.78,5,2019 - 06,88.000000,15.0,0.0,0.0,15.0,1.0,0.0,0.0
2494,68358.0,Big Data,Si,134.0,385.488507,51655.46,5,2018 - 08,93.636364,11.0,0.0,0.0,11.0,1.0,0.0,0.0


In [48]:
#Eliminar las columnas no necesarias
#df = df5.drop(["reducido","studentId","Minutos_Total","minutos_est","ciclo"],axis = 1)
## Pruebas con más columnas
df = df5.drop(["reducido","studentId"],axis = 1)
df.head(15)


,Graduado,Dias_Conectado,Minutos_Promedio,Minutos_Total,num_modulos,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas,prop_tareas_puntuales,prop_tareas_retrasadas,prop_tareas_no_entregadas
0,No,28.0,371.757500,10409.21,5,2019 - 08,71.288667,13.0,2.0,0.0,15.0,0.866667,0.000000,0.133333
1,Si,182.0,328.250934,59741.67,4,2019 - 05,90.000000,10.0,0.0,0.0,10.0,1.000000,0.000000,0.000000
2,Si,118.0,244.264661,28823.23,5,2019 - 05,90.606364,11.0,0.0,0.0,11.0,1.000000,0.000000,0.000000
3,Si,32.0,543.530313,17392.97,5,2019 - 06,88.000000,9.0,1.0,0.0,10.0,0.900000,0.000000,0.100000
4,No,14.0,308.148571,4314.08,5,2019 - 08,25.454545,3.0,8.0,0.0,11.0,0.272727,0.000000,0.727273
5,No,14.0,142.570000,1995.98,4,2017 - 06,23.181818,4.0,6.0,1.0,11.0,0.363636,0.090909,0.545455
6,No,39.0,192.296667,7499.57,5,2019 - 05,83.636364,11.0,0.0,0.0,11.0,1.000000,0.000000,0.000000
7,Si,109.0,243.157431,26504.16,5,2019 - 03,88.428667,15.0,0.0,0.0,15.0,1.000000,0.000000,0.000000
8,No,5.0,556.692000,2783.46,5,2019 - 01,6.666667,1.0,8.0,0.0,9.0,0.111111,0.000000,0.888889
9,No,77.0,183.567273,14134.68,5,2019 - 05,94.545455,11.0,0.0,0.0,11.0,1.000000,0.000000,0.000000


In [49]:
# Información acerca de las columnas del dataset unido
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 14 columns):
Graduado                     2500 non-null object
Dias_Conectado               2500 non-null float64
Minutos_Promedio             2500 non-null float64
Minutos_Total                2500 non-null float64
num_modulos                  2500 non-null int64
ciclo                        2500 non-null object
Calificacion_Promedio        2500 non-null float64
Tareas_Puntuales             2500 non-null float64
Tareas_No_Entregadas         2500 non-null float64
Tareas_Retrasadas            2500 non-null float64
Total_Tareas                 2500 non-null float64
prop_tareas_puntuales        2500 non-null float64
prop_tareas_retrasadas       2500 non-null float64
prop_tareas_no_entregadas    2500 non-null float64
dtypes: float64(11), int64(1), object(2)
memory usage: 293.0+ KB


In [50]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Dias_Conectado,2500.0,49.085600,33.072075,1.0,23.000000,42.000000,71.000000,199.000000
Minutos_Promedio,2500.0,259.036880,145.739734,0.0,158.080298,231.097917,332.742845,1173.203571
Minutos_Total,2500.0,12190.417404,9760.184061,0.0,5080.940000,10019.110000,16644.392500,107245.770000
num_modulos,2500.0,4.801600,0.536612,4.0,4.000000,5.000000,5.000000,7.000000
Calificacion_Promedio,2500.0,71.446212,28.357020,0.0,65.667000,82.777083,90.748141,100.000000
Tareas_Puntuales,2500.0,8.826800,4.105063,0.0,7.000000,9.000000,11.000000,29.000000
Tareas_No_Entregadas,2500.0,2.165200,3.795428,0.0,0.000000,0.000000,2.000000,34.000000
Tareas_Retrasadas,2500.0,0.182800,0.651732,0.0,0.000000,0.000000,0.000000,9.000000
Total_Tareas,2500.0,11.174800,3.232031,5.0,9.000000,10.000000,13.000000,34.000000
prop_tareas_puntuales,2500.0,0.800420,0.305287,0.0,0.769231,0.933333,1.000000,1.000000


De la información de arriba ud puede observar que hay valores Null/NaN en algunas de las columnas.

Para que nuestro modelo quede bien entrenado necesitamos procesar estos valores nulos de una forma adecuada.

Usted escogera la mejor estrategia como parte del desafío, pero en la siguiente celda encuentra un **ejemplo** the como puede hacer este procesamiento usanto la libreria *scikit-learn*.

<hr>

## Pre-procesando el dataset antes de entrenar

### Eliminando columnas no desadas

Vamos a **demostrar** acontinuación como usar el metodo DataFrame.drop().

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [51]:
df.tail()

,Graduado,Dias_Conectado,Minutos_Promedio,Minutos_Total,num_modulos,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas,prop_tareas_puntuales,prop_tareas_retrasadas,prop_tareas_no_entregadas
2495,No,56.0,162.077500,9076.34,6,2019 - 08,79.457000,10.0,0.0,0.0,10.0,1.0,0.0,0.0
2496,Si,36.0,446.748333,16082.94,5,2019 - 06,88.333571,14.0,0.0,0.0,14.0,1.0,0.0,0.0
2497,Si,71.0,131.249155,9318.69,4,2018 - 02,85.555556,9.0,0.0,0.0,9.0,1.0,0.0,0.0
2498,No,7.0,140.868571,986.08,5,2017 - 05,0.000000,0.0,10.0,0.0,10.0,0.0,0.0,1.0
2499,No,34.0,651.966765,22166.87,5,2019 - 08,78.527273,11.0,0.0,0.0,11.0,1.0,0.0,0.0


In [52]:
##Pruebs con más columnas
## df2 = df.drop(columns=['prop_tareas_puntuales','prop_tareas_retrasadas','prop_tareas_no_entregadas'], inplace=False)

In [53]:
#df2.tail()

<hr>

## Entrenando un clasificador


### Seleccionando FEATURES y definiendo la variable TARGET

In [54]:
df.columns

Index(['Graduado', 'Dias_Conectado', 'Minutos_Promedio', 'Minutos_Total',
       'num_modulos', 'ciclo', 'Calificacion_Promedio', 'Tareas_Puntuales',
       'Tareas_No_Entregadas', 'Tareas_Retrasadas', 'Total_Tareas',
       'prop_tareas_puntuales', 'prop_tareas_retrasadas',
       'prop_tareas_no_entregadas'],
      dtype='object')

In [55]:
features = df[
    [ 'Dias_Conectado', 'Minutos_Promedio', 'Minutos_Total',
       'num_modulos', 'Calificacion_Promedio', #'Tareas_Puntuales',
       #'Tareas_No_Entregadas', 'Tareas_Retrasadas', 'Total_Tareas',
       'prop_tareas_puntuales', 'prop_tareas_retrasadas',
       'prop_tareas_no_entregadas'
    ]
]
target = df3['Graduado']  ## No cambie la variable target!

### Dividiendo nuestro dataset en set de Entrenamiento y Pruebas

In [56]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=133)

### Escalando datos: MinMaxScaler

Nótese que se aplica una vez se ha separado el dataset en train y test para evitar *data leaking*

In [57]:
from sklearn.preprocessing import MinMaxScaler

#sc = MinMaxScaler()
#sc.fit(X_train)

### Escalando datos: StandardScaler

Nótese que se aplica una vez se ha separado el dataset en train y test para evitar *data leaking*

In [58]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)

StandardScaler()

In [59]:
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

### Entrenando un modelo ``DecisionTreeClassifier()`` para hacer feature selection

In [60]:
# Método para creacion de modelos basados en arbol de desición
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier(max_depth=15).fit(X_train, y_train) #Accuracy=0.730\nPrecision=0.730\nRecall=0.733\nF1 Score=0.7316103379721671

In [61]:
dtc.feature_importances_ 

array([0.45825192, 0.11633068, 0.10956311, 0.02450641, 0.22388852,
       0.03485619, 0.01402047, 0.0185827 ])

### Entrenando un modelo ``LogisticRegression()`` --Buen candidato, fácilmente interpretable

In [62]:
from sklearn.linear_model import LogisticRegression
#dtc = LogisticRegression().fit(X_train, y_train) # Accuracy=0.778\nPrecision=0.769\nRecall=0.797\nF1 Score=0.7827788649706457   
dtc = LogisticRegression(penalty='l1',solver='liblinear').fit(X_train, y_train) # Accuracy=0.782\nPrecision=0.775\nRecall=0.797\nF1 Score=0.7858546168958742'

In [63]:
dtc.coef_ 

array([[ 1.91189656,  0.268999  , -0.61692421,  0.06211074,  1.73940532,
         0.08971283, -0.07190007,  0.        ]])

### Entrenando un modelo ``SVM()``

In [64]:
from sklearn import svm
#dtc = svm.SVC().fit(X_train,y_train) #Accuracy=0.780\nPrecision=0.760\nRecall=0.821\nF1 Score=0.789272030651341'
dtc = svm.LinearSVC().fit(X_train,y_train) #Accuracy=0.784\nPrecision=0.778\nRecall=0.797\nF1 Score=0.7874015748031495')
#dtc = svm.NuSVC().fit(X_train,y_train) #Accuracy=0.780\nPrecision=0.764\nRecall=0.813\nF1 Score=0.7876447876447876

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [65]:
dtc.coef_ #For linear kernel

array([[ 0.68615831,  0.09688462, -0.23267191,  0.02492911,  0.6710386 ,
        -0.02512873, -0.03912414,  0.03368072]])

### Grid Search

In [66]:
from sklearn.metrics import f1_score ,make_scorer
from sklearn.model_selection import GridSearchCV

f1_scorer = make_scorer(f1_score,pos_label='No',greater_is_better=True)

#classifier = RandomForestClassifier()
#hyper_param_grid = {'n_estimators': [20,100,300,500], 
#                    'max_depth': [20,50,100], 
#                    'max_features': ['sqrt'],
#                    'min_samples_split': [2,5,10],
#                    }

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
#dtc = GridSearchCV( classifier, 
#                           hyper_param_grid, 
#                           scoring = f1_scorer,
#                           cv = 10, 
#                           n_jobs = -1,
#                           verbose = 3)
#Accuracy=0.784\nPrecision=0.772\nRecall=0.809\nF1 Score=0.7898832684824902
#Accuracy=0.774\nPrecision=0.767\nRecall=0.789\nF1 Score=0.7779960707269156

In [67]:
#classifier = RandomForestClassifier()
#hyper_param_grid = {'n_estimators': [20,100,300,500], 
#                    'max_depth': [20,50,100], 
#                    'max_features': ['sqrt'],
#                    'min_samples_split': [2,5,10],
#                    }

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
#dtc = GridSearchCV( classifier, 
#                           hyper_param_grid, 
#                           scoring = f1_scorer,
#                           cv = 10, 
#                           n_jobs = -1,
#                           verbose = 3)
#Accuracy=0.784\nPrecision=0.772\nRecall=0.809\nF1 Score=0.7898832684824902
#Accuracy=0.774\nPrecision=0.767\nRecall=0.789\nF1 Score=0.7779960707269156

In [68]:
#dtc.best_score_

In [69]:
#parameters = {'n_neighbors':[1, 21, 2],'weights':['uniform', 'distance'],'metric': ['euclidean', 'manhattan', 'minkowski']}
#classifier = KNeighborsClassifier()
#dtc = GridSearchCV(classifier, parameters,scoring = f1_scorer,
#                           cv = 10, 
#                           n_jobs = -1,
#                           verbose = 3)
#Accuracy=0.768\nPrecision=0.747\nRecall=0.813\nF1 Score=0.7786259541984732

In [70]:
#dtc.best_score_

In [71]:
parameters = {'kernel':('linear', 'rbf','sigmoid'), 'C':[1, 10]}
classifier = svm.SVC()
dtc = GridSearchCV(classifier, parameters,scoring = f1_scorer,
                           cv = 10, 
                           n_jobs = -1,
                           verbose = 3)
#Accuracy=0.782\nPrecision=0.780\nRecall=0.789\nF1 Score=0.7841584158415842'
#Accuracy=0.782\nPrecision=0.780\nRecall=0.789\nF1 Score=0.7841584158415842'

In [72]:
dtc.fit(X_train, y_train)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    5.4s finished


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf', 'sigmoid')},
             scoring=make_scorer(f1_score, pos_label=No), verbose=3)

In [73]:
dtc.best_score_

0.7986712019068472

In [74]:
dtc.best_params_

{'C': 1, 'kernel': 'linear'}

In [75]:
dtc.best_estimator_

SVC(C=1, kernel='linear')

### Haciendo predicciones del Sample Test

In [76]:
y_pred = dtc.predict(X_test)
print(y_pred)

['Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'Si'
 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si'
 'Si' 'Si' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'Si'
 'No' 'Si' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'Si'
 'No' 'No' 'Si' 'No' 'No' 'Si' 'No' 'No' 'Si' 'Si' 'No' 'No' 'Si' 'Si'
 'No' 'No' 'No' 'No' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'No' 'No' 'No'
 'Si' 'No' 'No' 'No' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'Si' 'No' 'Si' 'Si'
 'Si' 'No' 'Si' 'Si' 'Si' 'Si' 'No' 'No' 'Si' 'No' 'No' 'No' 'Si' 'Si'
 'No' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'Si' 'No' 'No' 'No' 'No' 'No'
 'No' 'No' 'No' 'No' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si'
 'No' 'No' 'Si' 'No' 'No' 'No' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'Si'
 'No' 'No' 'Si' 'Si' 'No' 'No' 'No' 'Si' 'No' 'Si' 'Si' 'Si' 'No' 'Si'
 'No' 'No' 'Si' 'No' 'Si' 'Si' 'No' 'No' 'No' 'Si' 'No' 'Si' 'Si' 'No'
 'No' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'Si' 'No' 'No'
 'No' 

### Analice la calidad del modelo a través de la matriz de confusión

In [77]:
!pip install seaborn --upgrade

     |████████████████████████████████| 225kB 8.6MB/s eta 0:00:01
  Found existing installation: seaborn 0.9.0
    Uninstalling seaborn-0.9.0:
      Successfully uninstalled seaborn-0.9.0


In [78]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

cf_matrix = confusion_matrix(y_test, y_pred)
group_names = ['`Positivo` Correto', '`Negativo` Errado', 'Falso `Positivo`', '`Negativo` Correto']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names, group_counts, group_percentages)]
labels = np.asarray(labels).reshape(2,2)
accuracy  = np.trace(cf_matrix) / float(np.sum(cf_matrix))
precision = cf_matrix[1,1] / sum(cf_matrix[:,1])
recall    = cf_matrix[1,1] / sum(cf_matrix[1,:])
f1_score  = 2*precision*recall / (precision + recall)
sns.heatmap(cf_matrix, annot=labels, fmt="")
stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={}".format(accuracy, precision, recall, f1_score)
plt.ylabel('True label')
plt.xlabel('Predicted label' + stats_text)
# Accuracy=0.778\nPrecision=0.765\nRecall=0.805\nF1 Score=0.7844660194174756' -- MinMax

Text(0.5, 23.52222222222222, 'Predicted label\n\nAccuracy=0.782\nPrecision=0.780\nRecall=0.789\nF1 Score=0.7841584158415842')

<hr>

## Scoring de la data requerida para hacer la entrega de la solución

Para el envío, necesita clasificar el siguiente dataset. Para hacer eso, usted necesita reproducir los mismos pasos de pre-procesamiento para que el dataset este en la misma estructura del que usted uso para construir su modelo. Despues de clasificar este dataframe, esperamos que usted entregue un archivo csv con las 2499 filar y una columna 'Graduado' con su predicción. **No cambie el orden del archivo a predecir ni borre filas**

In [79]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset2/master/for_submission/ToBePredicted.csv
df_to_be_predicted = pd.read_csv(r'ToBePredicted.csv')
df_to_be_predicted.tail()

--2020-09-04 08:13:29--  https://raw.githubusercontent.com/vanderlei-test/dataset2/master/for_submission/ToBePredicted.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33892 (33K) [text/plain]
Saving to: ‘ToBePredicted.csv’

100%[======================================>] 33,892      --.-K/s   in 0.002s  

2020-09-04 08:13:29 (21.5 MB/s) - ‘ToBePredicted.csv’ saved [33892/33892]



,studentId,reducido,ciclo
995,79594.0,Diagn. Psicoped.,2019 - 05
996,69470.0,Ventas Estrat.,2018 - 08
997,72921.0,Big Data,2019 - 02
998,80608.0,Interv. Psicoped,2019 - 05
999,86966.0,Interv. Psicoped,2019 - 07


In [80]:
df_to_be_predicted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
studentId    1000 non-null float64
reducido     1000 non-null object
ciclo        1000 non-null object
dtypes: float64(1), object(2)
memory usage: 23.5+ KB


In [81]:
# Uniendo los dataset
df2 = pd.merge(df_to_be_predicted,df_tabla_conexiones,how='inner')
df3 = df2.merge(df_modulos,how = 'left')
df4 = df3.loc[:, ["studentId","reducido","Dias_Conectado","Minutos_Promedio", "Minutos_Total","num_modulos"]]
df5 = pd.merge(df4,df_prop_tareas,how = 'left')

# Eliminando la columnas categóricas, pues no nos aportan más info
df = df5.drop(["reducido","studentId"],axis = 1)

df.tail()

,Dias_Conectado,Minutos_Promedio,Minutos_Total,num_modulos,ciclo,Calificacion_Promedio,Tareas_Puntuales,Tareas_No_Entregadas,Tareas_Retrasadas,Total_Tareas,prop_tareas_puntuales,prop_tareas_retrasadas,prop_tareas_no_entregadas
995,47.0,253.747234,11926.12,5,2019 - 05,87.667000,10.0,0.0,0.0,10.0,1.000000,0.0,0.000000
996,18.0,147.032778,2646.59,5,2018 - 08,64.400909,9.0,2.0,0.0,11.0,0.818182,0.0,0.181818
997,26.0,310.511154,8073.29,5,2019 - 02,81.000000,10.0,0.0,0.0,10.0,1.000000,0.0,0.000000
998,79.0,160.656076,12691.83,5,2019 - 05,93.600000,10.0,0.0,0.0,10.0,1.000000,0.0,0.000000
999,30.0,435.222333,13056.67,5,2019 - 07,30.667000,5.0,5.0,0.0,10.0,0.500000,0.0,0.500000


Observando los features declarados a continuación, sabemos que el dataset ha ser evaluado esta en el mismo formato usado para entrenar nuestro árbol de decisión anteriormente.

```features = df3[
    [
        'Minutos_Total', 'minutos_est', 
      'factor_minutos', 'Calificacion_Promedio',
      'Tareas_Puntuales','Total_Tareas',
      'prop_tareas_puntuales', 'prop_tareas_no_entregadas'
    ]
]
target = df3['Graduado']  ## No cambie la variable target!```

In [82]:
y_pred = dtc.predict(sc.transform(df[
    [
      'Dias_Conectado', 'Minutos_Promedio', 'Minutos_Total',
       'num_modulos', 'Calificacion_Promedio', 
       'prop_tareas_puntuales', 'prop_tareas_retrasadas',
       'prop_tareas_no_entregadas'
    ]
]))
print(y_pred)

['Si' 'No' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'No' 'No' 'No' 'Si'
 'No' 'No' 'No' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'No' 'Si' 'Si' 'No' 'No'
 'Si' 'Si' 'Si' 'No' 'No' 'No' 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'No' 'Si'
 'Si' 'No' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'Si'
 'No' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'No' 'No' 'Si' 'No' 'No' 'No'
 'Si' 'Si' 'No' 'No' 'No' 'Si' 'Si' 'No' 'Si' 'No' 'No' 'Si' 'No' 'Si'
 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'Si' 'Si'
 'No' 'No' 'No' 'No' 'Si' 'No' 'Si' 'Si' 'No' 'No' 'No' 'No' 'No' 'No'
 'Si' 'No' 'Si' 'Si' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'Si' 'No' 'No' 'No'
 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'No'
 'Si' 'Si' 'Si' 'No' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'No' 'No' 'Si' 'Si'
 'No' 'Si' 'Si' 'No' 'Si' 'No' 'Si' 'No' 'Si' 'No' 'No' 'Si' 'No' 'Si'
 'Si' 'No' 'Si' 'No' 'No' 'Si' 'Si' 'Si' 'No' 'No' 'No' 'No' 'No' 'Si'
 'Si' 'Si' 'No' 'Si' 'Si' 'Si' 'Si' 'Si' 'Si' 'No' 'Si' 'Si' 'No' 'Si'
 'Si' 

### Guardando los resultados de la predicción en un archivo csv

In [83]:
np.savetxt("results.csv", y_pred, delimiter=",", fmt='%s')

In [84]:
project.save_data(file_name="results.csv", data=pd.read_csv("results.csv", header=None).to_csv(header=["TARGET"], index=False))

{'file_name': 'results.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'desafio6-donotdelete-pr-exlisxjcpggbpt',
 'asset_id': '387f5d6b-5940-478d-aaa7-30b98f79a6d3'}

<hr>

## ¡Felicitaciones!

Si todo fue ejecutado sin errores, usted ya tiene un modelo basado en classificacion binaria y puede descargar sus resultados para subirlos como csv!

Para enviar su solución, ve a la página:

# https://anahuac.maratona.dev
